# Cleaning the OPENWHYD datalog

In [2]:

import pandas as pd
import requests                         # for scraping title from yt site
from bs4 import BeautifulSoup as bs     # for retrieving info on the tracks from the youtube website.
import time                             # for timing how long a batch of requests to youtube has taken
import os                               # for ringing bell when large batch of requests to youtube has finished


In [45]:
# importer 700MB log: 

log = pd.read_csv('/Users/aidanairuser/Desktop/OPENWHYD/yt_playlog_0404.csv')


In [49]:
# regarder le log. NOTE - the order here is not in terms of user, it's in terms of the time stamp

print (log.shape)
print ("nb of users:", log.user.nunique())
print ("nb of songs:", log.song.nunique())

log.head(5)


(25739537, 3)
nb of users: 45904
nb of songs: 631348


,timestamp,user,song
0,1392387533,0,t1l8Z6gLPzo
1,1392387538,1,t1l8Z6gLPzo
2,1392387556,2,t1l8Z6gLPzo
3,1392387561,3,we5gzZq5Avg
4,1392387566,4,we5gzZq5Avg


# on manipule le log pour le nettoyer?

In [24]:
# on supprime la colonne 'timestamp' pour l'instant

log.drop('timestamp', axis=1, inplace=True)

In [25]:
# use a groupby to produce the dataset in the form 'user - song - number of plays'

flipped = pd.DataFrame(log.groupby ( ['user', 'song']).song.count ( )).add_suffix('_count').reset_index()
flipped.rename(columns = {'user':'user_id'}, inplace = True)

flipped.head(5)

,user_id,song,song_count
0,0,-1b82VmUjh0,6
1,0,-3j6W3IWzsk,4
2,0,-8P6U_80r7Y,2
3,0,-9jp7xgarT0,3
4,0,-BijuEGNfHU,11


In [26]:
# Il n'y a que 631_348 chansons à vérifier -- pas 8 millions -- tant qu'on vérifie pour chaque chanson une fois

len(flipped)             # flipped contient 8 million de lignes qui montre utilisateur-chanson-écouté combien de fois
flipped.song.nunique()   # mais il n'y a que 630K chansons différentes dans tout ca

631348

In [27]:
# on crée une liste des chansons uniques
song_list = flipped.song.unique()

# on crée un df depuis cette liste
checking = pd.DataFrame({'song':song_list})

# on rajoute une colonne vide pour noter 'ok' ou 'manque'
checking["link"] = ""

# on rajoute 'No' par défaut
checking['link'] = 'No'

# on regarde combien de lignes il y aura à vérifier
len(checking)

631348

# Quand on nettoie par étapes on va parfois reprendre en important une version antérieure qui est 'mi-vérifié'

In [5]:
# import an earlier incomplete version of the checking files that has already thousands of lines checked
# virer une colonne de trop (ancien index) 
# montrer 5 premières lignes et vérifier que la première c'est bien "-1b82VmUjh0"

checking = pd.read_csv('/Users/aidanairuser/Desktop/OPENWHYD/yt_log_checked_DONE_0_631047_0404.csv')
checking.drop('Unnamed: 0', axis=1, inplace=True)
checking.head(5)

,song,link
0,-1b82VmUjh0,yes
1,-3j6W3IWzsk,yes
2,-8P6U_80r7Y,yes
3,-9jp7xgarT0,yes
4,-BijuEGNfHU,yes


# On note 'Yes' chaque fois qu'un lien youtube rend les infos demandées

In [11]:
# length of df to be checked is 631_348

start_time = time.time()

nb_ok = 0
row_start = 80_765
row_finish = 80_770
rows_total = row_finish - row_start

for row in range(row_start,row_finish):   
    toon = checking.iat[row, 0]
    yt = "https://www.youtube.com/watch?v="
    url = yt.strip() + toon.strip()
    page = requests.get(url)
    soup = bs (page.text, 'html.parser')

    for item in soup.find_all('h1',{'class':'watch-title-container'}): 
        for post in item.find_all('span',{'class':'watch-title'}): 
            checking.iat[row, 1] = 'yes'     
            nb_ok = nb_ok + 1
            row = row + 1   
            
print (nb_ok, 'are ok of', rows_total)

elapsed = ((time.time() - start_time)/60)
print ("ran in", elapsed, "minutes")

os.system('afplay /System/Library/Sounds/purr.aiff')
os.system('afplay /System/Library/Sounds/submarine.aiff')


5 are ok of 5
ran in 0.05510545174280802 minutes


0

In [14]:
# vérifier une trache de la df, une fois les lignes / links vérifiés

checking.iloc[80_765:80_770,] 


,song,link
80765,29J2B2FIgt0,yes
80766,2ABNMRWr1Sg,yes
80767,2AYV5pmENJ4,yes
80768,2C-Ucxo1UHs,yes
80769,2C0OEVWKZv0,yes


In [52]:
# download the file with the 'Yes' burned on so we don't have to start from scratch next time we run this.

checking.to_csv("yt_log_checked_0_631047_0404.csv")


# Clean up the original 0404 log using the 'checking' file that now lists all good / bad links

In [31]:
# import the finished 'checking' file

checking = pd.read_csv('/Users/aidanairuser/Desktop/OPENWHYD/yt_log_checked_DONE_0_631047_0404.csv')

In [50]:
# CHECKING X LOG = CLEAN LOG

# make a list of all the songs that have a dead link then build new df of 0404_log without dead links

checking_neg_only = checking.drop(checking[checking.link == "yes"].index)  # df avec uniquement les lignes 'no' (liens morts)

neg_list = checking_neg_only["song"].tolist()  # créer une liste depuis ce df des liens morts

clean = log[~log.song.isin(neg_list)]  # créer df avec que les chansons qui ne sont pas dans la liste de liens morts

# clean_df = pd.DataFrame(clean.drop('timestamp', axis=1))  # virer la colonne 'timestamp' OPTIONAL 

In [54]:
# MAINTENANT exporter le log 0404 propre

clean.to_csv("0404_log_clean.csv")

In [57]:
# comparer

import os


print ('* REGISTER OF GOOD BAD LINKS OUT OF THE 600K SONGS')
print (len(checking))
print (os.path.getsize('/Users/aidanairuser/Desktop/OPENWHYD/yt_log_checked_DONE_0_631047_0404.csv'))

print ('* ORIGINAL 0404 LOG')
print (len(log))
print (os.path.getsize('/Users/aidanairuser/Desktop/OPENWHYD/yt_playlog_0404.csv'))

print ('* CLEAN 0404 LOG')
print (len(clean))
print (os.path.getsize('/Users/aidanairuser/Desktop/OPENWHYD/0404_log_clean.csv'))

* REGISTER OF GOOD BAD LINKS OUT OF THE 600K SONGS
631348
14296361
* ORIGINAL 0404 LOG
25739537
725392726
* CLEAN 0404 LOG
22185155
815876501


# On revient si on trouve un souci avec un lien en travaillant avec le nouveau df propre

https://www.youtube.com/watch?v=2C0OEVWKZv0
    
this came from the clean df -- the link shouldn't be dead   

In [15]:
checking = pd.read_csv('/Users/aidanairuser/Desktop/OPENWHYD/yt_log_checked_DONE_0_631047_0404.csv')
checking.drop('Unnamed: 0', axis=1, inplace=True)
checking.head(5)

,song,link
0,-1b82VmUjh0,yes
1,-3j6W3IWzsk,yes
2,-8P6U_80r7Y,yes
3,-9jp7xgarT0,yes
4,-BijuEGNfHU,yes


In [8]:
# on vérifie pour ce lien - la liste des bon liens dit qu'il est bon

checking[checking.song == "2C0OEVWKZv0"]

,song,link
80769,2C0OEVWKZv0,yes


In [16]:
checking[checking.song == "2C0OEVWKZv0"]

,song,link
80769,2C0OEVWKZv0,yes


# EXPLICATION: le lien n'est pas 'mort' où on n'a aucune info sur le clip. En fait, il est juste 'pas accessible dans mon pays'. Surtout la page du clip fournit un groupe et un titre. Et donc notre fonction a rendu 'Yes' au lieu de 'Non' (et c'est peut-être mieux si le lien est effectivement vivant quand on ira en Italie par exemple)

In [ ]:

# LOG OF TIME SPENT (usually a rate of 10 hours for 100K) and nb lost (about a sixth)

# 0 - 30_000: 25750 are ok of 30000 / ran in 305.8400292476018 minutes - 100 rows a minute
# 30_000 - 45_000: 12426 are ok of 15000 / ran in 153.5392414490382 minutes - 100 rows a minute
# 45_000 - 60_000: 12815 are ok of 15000 / ran in 150.01641809940338 minutes
# 60_000 - 70_000: 8564 are ok of 10000 / ran in 97.89230124553045 minutes
# 70_000 - 80_000: 8176 are ok of 10000 / ran in 93.95360571543375 minutes
# 80_000 - 90_000: 8281 are ok of 10000 / ran in 119.0353542526563 minutes
# 90_000 - 110_000: 16502 are ok of 20000 / ran in 210.82278453509014 minutes
# 110_00- 130_000: 16515 are ok of 20000 / ran in 211.21969944636027 minutes
# 130 to 170 : 32245 are ok of 40000 / ran in 395.40184058348336 minutes
# 170 to 200 : 24438 are ok of 30000 / ran in 298.89787494738897 minutes
# 200 to 210 : 8213 are ok of 10000 / ran in 107.48892344633738 minutes
# ran 210 to 220 in parts
# 220 to 225: 3922 are ok of 5000 / ran in 47.93766736984253 minutes
# 225 to 230: 3970 are ok of 5000 / ran in 49.198327732086184 minutes
# 230 to 240: 8369 are ok of 10000 / ran in 100.20720508098603 minutes
# 240 to 260: 17260 are ok of 20000 / ran in 207.02818228403729 minutes
# 260 to 280: 17438 are ok of 20000 / ran in 215.76910527944565 minutes
# 285 to 320: 30500 are ok of 35000 / ran in 342.78767923116686 minutes
# 320 to 340: 16298 are ok of 20000 / ran in 194.8126765847206 minutes
# 340 to 370: 24327 are ok of 30000 / ran in 301.23866998354595 minutes
# 370 to 400 : 24062 are ok of 30000 / ran in 312.1866168141365 minutes
# 400 to 410: 7960 are ok of 10000 / ran in 96.79436396360397 minutes
# 410 to 440: 23797 are ok of 30000 / ran in 286.8092731197675 minutes
# 440 to 455 : 11926 are ok of 15000 / ran in 143.93594185113906 minutes
# 455 to 475 : 15889 are ok of 20000 / ran in 195.29008544683455 minutes
# 475 - 490: 11903 are ok of 15000 / ran in 140.14434896707536 minutes
# 490 - 500
# 500 - 520 : 16027 are ok of 20000 / ran in 190.24522956609727 minutes
# 520 - 540 : 16067 are ok of 20000 / ran in 208.85930556853612 minutes
# 540 - 570: 23401 are ok of 30000 / ran in 274.56876204808555 minutes
# 570 - 590 : 16037 are ok of 20000 / ran in 182.28820083141326 minutes
# 590 - 600 : 8016 are ok of 10000 / ran in 95.35403951009114 minutes
# 600 - 630: 25182 are ok of 30000 / ran in 307.33169128894804 minutes